In [1]:
import tensorflow.compat.v1 as tf
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [2]:
# 랜덤에 의해 똑같은 결과를 재현하도록 시드 설정
# 하이퍼파라미터를 튜닝하기 위한 용도(흔들리면 무엇때문에 좋아졌는지 알기 어려움)
tf.set_random_seed(777)

In [3]:
# Standardization
def data_standardization(x):
    x_np = np.asarray(x)
    return (x_np - x_np.mean()) / x_np.std()
 
# 너무 작거나 너무 큰 값이 학습을 방해하는 것을 방지하고자 정규화한다
# x가 양수라는 가정하에 최소값과 최대값을 이용하여 0~1사이의 값으로 변환
# Min-Max scaling
def min_max_scaling(x):
    x_np = np.asarray(x)
    return (x_np - x_np.min()) / (x_np.max() - x_np.min() + 1e-7) # 1e-7은 0으로 나누는 오류 예방차원
 
# 정규화된 값을 원래의 값으로 되돌린다
# 정규화하기 이전의 org_x값과 되돌리고 싶은 x를 입력하면 역정규화된 값을 리턴한다
def reverse_min_max_scaling(org_x, x):
    org_x_np = np.asarray(org_x)
    x_np = np.asarray(x)
    return (x_np * (org_x_np.max() - org_x_np.min() + 1e-7)) + org_x_np.min()
 
 

In [4]:
# 하이퍼파라미터
input_data_column_cnt = 5  # 입력데이터의 컬럼 개수(Variable 개수)
output_data_column_cnt = 1 # 결과데이터의 컬럼 개수

seq_length = 28            # 1개 시퀀스의 길이(시계열데이터 입력 개수)
rnn_cell_hidden_dim = 20   # 각 셀의 (hidden)출력 크기
forget_bias = 1.0          # 망각편향(기본값 1.0)
num_stacked_layers = 1     # stacked LSTM layers 개수
keep_prob = 1.0            # dropout할 때 keep할 비율

epoch_num = 500            # 에폭 횟수(학습용전체데이터를 몇 회 반복해서 학습할 것인가 입력)
learning_rate = 0.01       # 학습률

In [5]:
#판다스이용 csv파일 로딩
raw_dataframe = pd.read_csv('data/medibloc(2018~2021).csv', encoding='cp949')
# 데이터 정보 출력
raw_dataframe.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   날짜      1095 non-null   object 
 1   시가      1095 non-null   float64
 2   고가      1095 non-null   float64
 3   저가      1095 non-null   float64
 4   종가      1095 non-null   float64
 5   거래량     1095 non-null   int64  
 6   시가총액    1095 non-null   int64  
dtypes: float64(4), int64(2), object(1)
memory usage: 60.0+ KB


In [6]:
# raw_dataframe.drop('Date', axis=1, inplace=True) # 시간열을 제거하고 dataframe 재생성하지 않기
del raw_dataframe['날짜'] # 위 줄과 같은 효과

In [7]:
raw_dataframe

,시가,고가,저가,종가,거래량,시가총액
0,16.74,17.17,15.52,16.02,692525441,47525866749
1,16.13,16.31,15.40,15.95,536859145,47306351604
2,15.92,17.05,15.87,16.69,459122028,49515243697
3,16.70,17.47,16.58,17.23,451311825,51098098705
4,17.26,18.31,16.59,17.14,565735320,50849761724
...,...,...,...,...,...,...
1090,102.73,119.43,102.72,111.63,229362666554,305655135725
1091,112.61,140.07,109.94,130.17,503803529611,356424611490
1092,130.17,277.06,127.18,273.11,2256586556340,747797814365
1093,267.53,398.55,267.53,298.93,1629150393198,818506251680


In [8]:
stock_info = raw_dataframe.values[1:].astype(np.float) # 금액&거래량 문자열을 부동소수점형으로 변환한다
print("stock_info.shape: ", stock_info.shape)
print("stock_info[0]: ", stock_info[0])
stock_info

stock_info.shape:  (1094, 6)
stock_info[0]:  [1.61300000e+01 1.63100000e+01 1.54000000e+01 1.59500000e+01
 5.36859145e+08 4.73063516e+10]


array([[1.61300000e+01, 1.63100000e+01, 1.54000000e+01, 1.59500000e+01,
        5.36859145e+08, 4.73063516e+10],
       [1.59200000e+01, 1.70500000e+01, 1.58700000e+01, 1.66900000e+01,
        4.59122028e+08, 4.95152437e+10],
       [1.67000000e+01, 1.74700000e+01, 1.65800000e+01, 1.72300000e+01,
        4.51311825e+08, 5.10980987e+10],
       ...,
       [1.30170000e+02, 2.77060000e+02, 1.27180000e+02, 2.73110000e+02,
        2.25658656e+12, 7.47797814e+11],
       [2.67530000e+02, 3.98550000e+02, 2.67530000e+02, 2.98930000e+02,
        1.62915039e+12, 8.18506252e+11],
       [2.99900000e+02, 3.27750000e+02, 2.72890000e+02, 2.86960000e+02,
        5.86518672e+11, 7.85724655e+11]])

In [9]:
price = stock_info[:,:-2]
norm_price = min_max_scaling(price)
print("price.shape: ", price.shape)
print("price[0]: ", price[0])
print("norm_price[0]: ", norm_price[0])
print("="*100) # 화면상 구분용

price.shape:  (1094, 4)
price[0]:  [16.13 16.31 15.4  15.95]
norm_price[0]:  [0.03621563 0.03666927 0.03437587 0.03576199]


In [10]:
volume = stock_info[:,-2:-1]
norm_volume = min_max_scaling(volume) # 거래량형태 데이터 정규화 처리
print("volume.shape: ", volume.shape)
print("volume[0]: ", volume[0])
print("norm_volume[0]: ", norm_volume[0])
print("="*100) # 화면상 구분용

volume.shape:  (1094, 1)
volume[0]:  [5.36859145e+08]
norm_volume[0]:  [0.00023791]


In [11]:
volume = stock_info[:,-1:]
norm_volume = min_max_scaling(volume) # 거래량형태 데이터 정규화 처리
print("volume.shape: ", volume.shape)
print("volume[0]: ", volume[0])
print("norm_volume[0]: ", norm_volume[0])
print("="*100) # 화면상 구분용

volume.shape:  (1094, 1)
volume[0]:  [4.73063516e+10]
norm_volume[0]:  [0.05202078]


In [12]:
x = np.concatenate((norm_price, norm_volume), axis=1) # axis=1, 세로로 합친다
print("x.shape: ", x.shape)
print("x[0]: ", x[0])    # x의 첫 값
print("x[-1]: ", x[-1])  # x의 마지막 값
print("="*100) # 화면상 구분용

x.shape:  (1094, 5)
x[0]:  [0.03621563 0.03666927 0.03437587 0.03576199 0.05202078]
x[-1]:  [0.75137982 0.82156808 0.68330855 0.71876811 0.959704  ]


In [13]:
y = x[:, [-2]] # 타켓은 주식 종가이다
print("y[0]: ",y[0])     # y의 첫 값
print("y[-1]: ",y[-1])   # y의 마지막 값

y[0]:  [0.03576199]
y[-1]:  [0.71876811]


In [14]:
dataX = [] # 입력으로 사용될 Sequence Data
dataY = [] # 출력(타켓)으로 사용

In [15]:
for i in range(0, len(y) - seq_length):
    _x = x[i : i+seq_length]
    _y = y[i + seq_length] # 다음 나타날 주가(정답)
    if i == 0:
        print(_x, "->", _y) # 첫번째 행만 출력해 봄
    dataX.append(_x) # dataX 리스트에 추가
    dataY.append(_y) # dataY 리스트에 추가

[[0.03621563 0.03666927 0.03437587 0.03576199 0.05202078]
 [0.03568638 0.03853424 0.03556037 0.03762696 0.05473601]
 [0.03765216 0.03959273 0.03734973 0.03898788 0.05668169]
 [0.03906348 0.04170972 0.03737493 0.03876106 0.05637643]
 [0.03866025 0.03984475 0.03795459 0.03931551 0.05716698]
 [0.03941632 0.04115527 0.03908869 0.04115527 0.05982989]
 [0.04090325 0.04614532 0.03767736 0.04569168 0.06639612]
 [0.04554046 0.04874115 0.04440636 0.04672497 0.0679076 ]
 [0.04685098 0.05466368 0.04685098 0.0515638  0.07489692]
 [0.05148819 0.05443686 0.05148819 0.05395801 0.07836315]
 [0.05390761 0.05395801 0.04853953 0.05032889 0.073088  ]
 [0.0504045  0.05118577 0.04904357 0.04909398 0.07131616]
 [0.04906878 0.05426044 0.04894277 0.05277351 0.07664981]
 [0.05284911 0.0559994  0.05053051 0.05317674 0.07722696]
 [0.05287432 0.06981023 0.0527231  0.06981023 0.10129861]
 [0.06988583 0.0930719  0.06988583 0.0930719  0.13496604]
 [0.09662542 0.11366214 0.09455883 0.09478565 0.13742462]
 [0.09392878 0

In [16]:
# 학습용/테스트용 데이터 생성
# 전체 70%를 학습용 데이터로 사용
train_size = int(len(dataY) * 0.8)
# 나머지(30%)를 테스트용 데이터로 사용
test_size = len(dataY) - train_size

In [17]:
# 데이터를 잘라 학습용 데이터 생성
trainX = np.array(dataX[0:train_size])
trainY = np.array(dataY[0:train_size])

In [18]:
# 데이터를 잘라 테스트용 데이터 생성
testX = np.array(dataX[train_size:len(dataX)])
testY = np.array(dataY[train_size:len(dataY)])

In [19]:
# 텐서플로우 플레이스홀더 생성
# 입력 X, 출력 Y를 생성한다
tf.disable_eager_execution()
X = tf.placeholder(tf.float32, [None, seq_length, input_data_column_cnt])
print("X: ", X)
Y = tf.placeholder(tf.float32, [None, 1])
print("Y: ", Y)

X:  Tensor("Placeholder:0", shape=(None, 28, 5), dtype=float32)
Y:  Tensor("Placeholder_1:0", shape=(None, 1), dtype=float32)


In [20]:
# 검증용 측정지표를 산출하기 위한 targets, predictions를 생성한다
targets = tf.placeholder(tf.float32, [None, 1])
print("targets: ", targets)
 
predictions = tf.placeholder(tf.float32, [None, 1])
print("predictions: ", predictions)

targets:  Tensor("Placeholder_2:0", shape=(None, 1), dtype=float32)
predictions:  Tensor("Placeholder_3:0", shape=(None, 1), dtype=float32)


In [21]:
# 모델(LSTM 네트워크) 생성
def lstm_cell():
    # LSTM셀을 생성
    # num_units: 각 Cell 출력 크기
    # forget_bias:  to the biases of the forget gate 
    #              (default: 1)  in order to reduce the scale of forgetting in the beginning of the training.
    # state_is_tuple: True ==> accepted and returned states are 2-tuples of the c_state and m_state.
    # state_is_tuple: False ==> they are concatenated along the column axis.
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_cell_hidden_dim, 
                                        forget_bias=forget_bias, state_is_tuple=True, activation=tf.nn.softsign)
    if keep_prob < 1.0:
        cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
    return cell

In [24]:
# num_stacked_layers개의 층으로 쌓인 Stacked RNNs 생성
tf.disable_v2_behavior()
stackedRNNs = [lstm_cell() for _ in range(num_stacked_layers)]
multi_cells = tf.contrib.rnn.MultiRNNCell(stackedRNNs, state_is_tuple=True) if num_stacked_layers > 1 else lstm_cell()
 

AttributeError: module 'tensorflow.compat.v1' has no attribute 'contrib'

In [ ]:
# RNN Cell(여기서는 LSTM셀임)들을 연결
hypothesis, _states = tf.nn.dynamic_rnn(multi_cells, X, dtype=tf.float32)
print("hypothesis: ", hypothesis)

In [ ]:
# [:, -1]를 잘 살펴보자. LSTM RNN의 마지막 (hidden)출력만을 사용했다.
# 과거 여러 거래일의 주가를 이용해서 다음날의 주가 1개를 예측하기때문에 MANY-TO-ONE형태이다
hypothesis = tf.contrib.layers.fully_connected(hypothesis[:, -1], output_data_column_cnt, activation_fn=tf.identity)